In [1]:
import requests
from bs4 import BeautifulSoup

In [10]:
#using header as amazon dont allow scrapping the data
header={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'}
url='https://www.amazon.in/s?k=mobile+phones&rh=n%3A21775353031&ref=nb_sb_noss' #mobile phones link in amazon

search_response=requests.get(url = url, headers=header) #get the link response
#search_response.status_code #should return 200 for succesful response
soup=BeautifulSoup(search_response.content) #pass it tp beautiful soup for parsing

In [13]:
product_names=[]
for i in soup.findAll("span",{'class':'a-size-medium a-color-base a-text-normal'}): # the tag which is common for all the names of products
    product_names.append(i.text) #adding the product names to the list

In [52]:
products = product_names[6:16] #taking 10 mobile phones
print("10 mobies names :")
products

10 mobies names :


['OnePlus 8 (Onyx Black 8GB RAM+128GB Storage)',
 'OnePlus 8 Pro (Ultramarine Blue 12GB RAM+256GB Storage)',
 'Samsung Galaxy M01 (Black, 3GB RAM, 32GB Storage) with No Cost EMI/Additional Exchange Offers',
 'Redmi 8A Dual (Sea Blue, 3GB RAM, 32GB Storage) – Dual Cameras & 5,000 mAH Battery',
 'Samsung Galaxy M31s (Mirage Blue, 6GB RAM, 128GB Storage)',
 'Samsung Galaxy M21 (Midnight Blue, 4GB RAM, 64GB Storage)',
 'Samsung Galaxy M01 (Blue, 3GB RAM, 32GB Storage) with No Cost EMI/Additional Exchange Offers',
 'Redmi 8A Dual (Sky White, 2GB RAM, 32GB Storage) – Dual Cameras & 5,000 mAH Battery',
 'Samsung Galaxy M21 (Raven Black, 4GB RAM, 64GB Storage)',
 'Samsung Galaxy M01 Core (Blue, 1GB RAM, 16GB Storage) with No Cost EMI/Additional Exchange Offers']

In [17]:
#each product in amazon is identified by unique number asin, so lets get that numbers so that we can use those numbers to get the respective mobile page and reviews
data_asin=[]
for i in soup.findAll("div",{'class' : "sg-col-20-of-24 s-result-item s-asin sg-col-0-of-12 sg-col-28-of-32 sg-col-16-of-20 sg-col sg-col-32-of-36 sg-col-12-of-16 sg-col-24-of-28"}): # the tag which is common for all the names of products
    data_asin.append(i['data-asin']) #adding the product names to the list

In [53]:
asin_numbers = data_asin[6:16] #taking asin of those above mobiles
print("ASIN numbers of these 10 mobiles:")
asin_numbers

ASIN numbers of these 10 mobiles:


['B071Z97T2C',
 'B077Q42GDF',
 'B086KCCMCP',
 'B07WPVLKPW',
 'B07DJCJBB3',
 'B07HGJJ559',
 'B086KCDGTQ',
 'B07X1KSWZ3',
 'B07HGH8D2R',
 'B089MS3GLM']

In [19]:
#using the asin numbers get the "all customer reviews" link of each mobile
link=[]
for i in range(len(asin_numbers)):
    response=requests.get(url = "https://www.amazon.in/dp/" + asin_numbers[i] + "#customerReviews", headers = header)
    soup=BeautifulSoup(response.content)
    for i in soup.findAll("a",{'data-hook':"see-all-reviews-link-foot"}): #get the links
        link.append(i['href'])

In [20]:
link #links of "all customer reviews" of each mobile

['/Test-Exclusive-237/product-reviews/B071Z97T2C/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
 '/Test-Exclusive-545/product-reviews/B077Q42GDF/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
 '/Samsung-Galaxy-Storage-Additional-Exchange/product-reviews/B086KCCMCP/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
 '/Redmi-8A-Dual-Blue-Storage/product-reviews/B07WPVLKPW/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
 '/Test-Exclusive-712/product-reviews/B07DJCJBB3/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
 '/Test-Exclusive-614/product-reviews/B07HGJJ559/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
 '/Samsung-Galaxy-Storage-Additional-Exchange/product-reviews/B086KCDGTQ/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
 '/Redmi-8A-Dual-White-Storage/product-reviews/B07X1KSWZ3/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
 '/Test-Exclusive-667/product-reviews/B

In [22]:
#we take the above links and get the first page reviews of each mobile
reviews=[]
for j in range(len(link)): #for each reviews page link
    for k in range(1): #page number
        response=requests.get(url ="https://www.amazon.in" + link[j]+'&pageNumber='+str(k), headers = header)
        soup=BeautifulSoup(response.content)
        for i in soup.findAll("span",{'data-hook':"review-body"}): #get the reviews
            reviews.append(i.text)

In [50]:
print("total number of reviews scrapped: {} : 10 reviews each per mobile".format(len(reviews)))

total number of reviews scrapped: 100 : 10 reviews each per mobile


In [35]:
import numpy as np
import pandas as pd
mobiles = np.repeat(products,10) #repeating the mobile name for each review to make the final data
mobiles_reviews = pd.DataFrame(list(zip(mobiles,reviews)), columns =['mobile', 'review']) #dataframe of reviews
mobiles_reviews.to_csv("scrapped_reviews.csv", index=False) #csv file of reviews

100